# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
B. Saha  ->  B. Saha  |  ['B. Saha']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
R. V  ->  R. v. Boekel  |  ['R. v']
Arxiv has 85 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.18748


extracting tarball to tmp_2505.18748...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.18939


extracting tarball to tmp_2505.18939...

...

extracting tarball to tmp_2505.19875... done.


G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure plots/Sun_Fit.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{plots/Sun_Fit.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure plots/Kiel_diagram.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{plots/Kiel_diagram.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.

Found 97 bibliographic references in tmp_2505.19875/aa53526-24.bbl.
Retrieving document from  https://arxiv.org/e-print/2505.20061


/tmp/ipykernel_3408/2822249172.py:52: LatexWarning: 2505.20061 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.19832-b31b1b.svg)](https://arxiv.org/abs/2505.19832) | **Reconciling extragalactic star formation efficiencies with theory: insights from PHANGS**  |
|| S. E. Meidt, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark> |
|*Appeared on*| *2025-05-27*|
|*Comments*| *Accepted for publication in A&A, 28 pages, 8 figures*|
|**Abstract**|            New extragalactic measurements of the cloud population-averaged star formation (SF) efficiency per freefall time $\rm\epsilon_{\rm ff}$ from PHANGS show little sign of theoretically predicted dependencies on cloud-scale virial level or velocity dispersion. We explore ways to bring theory into consistency with observations, highlighting systematic variations in internal density structure that must happen together with an increase in virial level typical towards galaxy centers. To introduce these variations into conventional turbulence-regulated SF models we adopt three adjustments motivated by the host galaxy's influence on the cloud-scale: we incorporate self-gravity and a gas density distribution that contains a broad power-law (PL) component and resembles the structure observed in local resolved clouds, we let the internal gas kinematics include motion in the background potential and let this regulate the onset of self-gravitation, and we assume that the gas density distribution is in a steady-state for only a fraction of a freefall time. The combined result is a strong reduction to $\rm\epsilon_{\rm ff}$ predicted in multi-freefall (MFF) scenarios compared to purely lognormal probability density functions and variations that are tied to the PL slope $\alpha$. The $\alpha$ needed to match PHANGS $\rm\epsilon_{\rm ff}$'s vary systematically with environment in the sense that gas sitting furthest from virial balance contains more gas at high density. With this `galaxy regulation' behavior included, our `self-gravitating' sgMFF models function similar to the original, roughly `virialized cloud' single-freefall models. However, outside disks with their characteristic regulation, the flexible MFF models may be better suited.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.19875-b31b1b.svg)](https://arxiv.org/abs/2505.19875) | **Impact of <3D> NLTE on GCE of oxygen with the RAdial Velocity Experiment**  |
|| <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-05-27*|
|*Comments*| *11 pages, 12 figures. Accepted for publication in A&A*|
|**Abstract**|            Stellar abundances, coupled with kinematics are a unique way to understand the chemo-dynamical processes that occurred to build the Milky Way and its local volume as we observe today. However, measuring abundances is challenging as one needs to properly address the effect of departure from the Local Thermodynamic Equilibrium (LTE), as well as the commonly used 1-dimensional model atmosphere. In this work, we constrain the chemical evolution of [O/Fe] in FG stars of the RAVE survey with [O/Fe] abundances derived in non-LTE (NLTE) and with horizontally-temporally-averaged 3D (<3D>) model atmospheres. Using standard spectral fitting method, we determine for the first time LTE and NLTE [O/Fe] ratios from the O triplet at 8446A in turn-off and dwarf stars thanks to intermediate-resolution RAVE spectra, assuming both 1D and <3D> model atmosphere. NLTE effects play a significant role when determining oxygen even at a resolution of R= 7500. Typical NLTE-LTE corrections of the order of -0.12 dex are measured in dwarfs and turn-off stars using 1D MARCS models. In contrast to applying <3D> NLTE abundance corrections or the classical 1D LTE, the full <3D> NLTE spectral fitting yields improving the precision of abundances by nearly 10%. We show that the decrease of [O/Fe] in the super-solar [Fe/H] regime is rather characterised by a flat trend when [O/Fe] is computed in <3D> NLTE from full spectral fitting. We attribute this flattening at super-solar [Fe/H] to the interplay between locally born stars with negative [O/Fe] and stars migrated from the inner MW regions with super-solar [O/Fe], supporting the complex chemo-dynamical history of the Solar neighbourhood. Our results are key for understanding the effects of <3D> and NLTE when measuring [O/Fe]. This work is a test bed for the analysis of 4MOST low-resolution spectra that will share similar properties as RAVE in the red wavelength domain.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.19583-b31b1b.svg)](https://arxiv.org/abs/2505.19583) | **Identifying lopsidedness in spiral galaxies using a Deep Convolutional Neural Network**  |
|| <mark>B. Saha</mark>, S. Sarkar, A. Banerjee |
|*Appeared on*| *2025-05-27*|
|*Comments*| *19 pages, 9 figures, 7 tables*|
|**Abstract**|            About 30\% of disk galaxies show lopsidedness in their stellar disk. Although such a large-scale asymmetry in the disk can be primarily looked upon as a long-lived mode ($m=1$), the physical origin of the lopsidedness in the disk continues to be a puzzle. In this work, we develop an automated approach to identify lopsided galaxies from the SDSS DR18 using a Deep Convolutional Neural Network (DCNN) based on the publicly available AlexNet architecture. We select nearly face-on spiral galaxies from SDSS DR18 with the Petrosian 90\% light radius (\textit{petroR90\_i}) greater than $20^{''}$. Based on the visual inspection, we choose 106 lopsided spiral galaxies and 105 symmetric spiral galaxies, as our training set. Our trained model achieves a testing accuracy of 92.8\% at the end of 150 epochs. We then employ the trained model on a set of 813 face-on spiral galaxies from SDSS DR18 with $17^{''} \le petroR90\_i \le 20^{''} $ and identify 452 new lopsided spiral galaxies. We next investigate the cosmic web environments in which the galaxies are located, using the Hessian matrix of the density field. We find that 39\% of the lopsided galaxies are located in sparser environments such as sheets and voids. This may provide interesting clues towards understanding the origin of lopsidedness in isolated galaxies, where distortion due to the tidal interactions is less frequent.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.18748-b31b1b.svg)](https://arxiv.org/abs/2505.18748) | **Clustering analysis of BOSS-CMASS galaxies with semi-analytical model for galaxy formation and halo occupation distribution**  |
|| Z. Zhai, A. Benson, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-27*|
|*Comments*| *15 pages, 10 pages, 2 tables; comments welcome*|
|**Abstract**|            The spatial distribution and clustering property of massive and luminous galaxies have provided important constraints on the fundamental cosmological parameters and physical processes governing galaxy formation. In this work, we construct and compare independent galaxy-halo connection models in the application of clustering measurement at non-linear scales of BOSS-CMASS galaxies. In particular, we adopt a halo occupation distribution (HOD) model with 11 parameters and a semi-analytical model (SAM) with 16 parameters to describe the galaxy correlation function including the projected correlation function, redshift space monopole and quadrupole. We focus on the redshift space distortion effect caused by the galaxy peculiar velocity. With a empirical model for the velocity field and the emulator technique, we can explore the parameter space of both models. We find that the HOD model is able to recover the underlying velocity field of SAM with an accuracy of 3\%, and can be improved to 1\% when the analysis is restricted to scales above 1$h^{-1}$Mpc. The comparison is based on multiple samplings in the parameter space which can verify the convergence of the empirical and physical model for galaxy formation. Then we perform constraints on the model parameters using clustering measurement of CMASS galaxies. Although limited by the emulator accuracy and the flexibility of the model, we find that the clustering measurement is capable of constraining a subset of the SAM parameters, especially for components sensitive to the star formation rate. This result leads us to anticipate that a joint analysis of both clustering and abundance measurements can significantly constrain the parameters of galaxy formation physics, which requires further investigation from both theoretical and observational aspects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.18939-b31b1b.svg)](https://arxiv.org/abs/2505.18939) | **Ground Calibration Result of the Wide-field X-ray Telescope (WXT) onboard the Einstein Probe**  |
|| H. Cheng, et al. -- incl., <mark>J. Li</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-27*|
|*Comments*| *28 pages, 18 figures, 2 tables. submitted to Experimental Astronomy*|
|**Abstract**|            We report on results of the on-ground X-ray calibration of the Wide-field X-ray Telescope (WXT) built from novel lobster-eye micro-pore optics, onboard the Einstein Probe (EP) satellite. To fully characterize the instrumental performance and properties, a series of tests and calibrations have been carried out at different levels of devices, assemblies and the complete module before the launch of EP. In this paper, we present the calibration results of three flight model modules (FM1, FM5 and FM11) obtained during their end-to-end module calibration experiments carried out at the 100-m X-ray Test Facility (100XF) of IHEP, CAS. Measurements of the Point Spread Function (PSF), effective area, and energy response were performed for multiple incident directions and several characteristic X-ray emission line energies. Specifically, the distributions of the PSF and effective areas are found to be roughly uniform across the FoV, in large agreement with the prediction of lobster-eye optics. Their energy dependence behavior aligns well with theoretical predictions and Monte Carlo simulations. At 1.25 keV, the full width at half maximum (FWHM) of the focal spot is in range of 3-7 arcmin (a median of 4.2) and the effective area in range of 2-3 $cm^2$. Noticeably, the flight model instruments demonstrate a $\sim1.5$ arcmin spatial resolution improvement over the previously launched Lobster Eye Imager for Astronomy. The properties of the complementary metal-oxide semiconductor (CMOS) sensors were also calibrated. The gain coefficients are in range of 6.4-6.9 eV/DN. The energy resolutions are in range of 120-140 eV at 1.25 keV, meeting design requirements. These calibration results have been ingested into the first version of calibration database (CALDB) and applied to the analysis of the scientific data acquired by WXT after the launch of EP.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.20061-b31b1b.svg)](https://arxiv.org/abs/2505.20061) | **X-Ray spectroscopy and timing (XSPECT) experiment on XPoSat -- instrument configuration and science prospects**  |
|| <mark>R. V</mark>, et al. |
|*Appeared on*| *2025-05-27*|
|*Comments*| *20 pages, 13 figures. Submitted to JATIS*|
|**Abstract**|            X-ray Polarimeter Satellite (XPoSat) with POLarimeter Instrument in X-rays (POLIX), is India's first spacecraft dedicated to study medium energy X-ray polarisation from celestial objects. X-Ray Spectroscopy and Timing (XSPECT) instrument on XPoSat is configured to study long term spectral behaviour of select sources in Soft X-ray regime. The instrument uses Swept Charge Devices (SCD)s to provide large area and spectral performance with passive cooling arrangement. The instrument consists of set of collimators with two different FOVs, optical light blocking filters, and signal processing electronics. The instrument was designed, tested and calibrated on ground. The unique opportunity is provided by ISRO's XPoSat mission, where a source is observed for longer duration. The device used also enables spectroscopy study of brighter sources compared to the CCD based spectrometers. The first results demonstrate instrument capability for spectral studies in the 0.8 keV-15 keV energy band.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.19832/./outplot2asingle.png', 'tmp_2505.19832/./PHANGSplot.jpeg', 'tmp_2505.19832/./outplot4.png']
copying  tmp_2505.19832/./outplot2asingle.png to _build/html/
copying  tmp_2505.19832/./PHANGSplot.jpeg to _build/html/
copying  tmp_2505.19832/./outplot4.png to _build/html/
exported in  _build/html/2505.19832.md
    + _build/html/tmp_2505.19832/./outplot2asingle.png
    + _build/html/tmp_2505.19832/./PHANGSplot.jpeg
    + _build/html/tmp_2505.19832/./outplot4.png
found figures []
exported in  _build/html/2505.19875.md


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{◦ee}{\ensuremath{^\circ}}$
$\newcommand{\kmskpc}{ap{km s^{-1} kpc^{-1}}}$
$\newcommand{\meanv}{\ensuremath{<v>}}$
$\newcommand{\python}{{\tt Python}}$
$\newcommand{\avir}{\mbox{\langle\alpha_{\rm vir}\rangle}}$
$\newcommand{\disp}{\mbox{\langle\sigma_{\rm mol}\rangle}}$
$\newcommand{\eff}{\mbox{\langle\epsilon_{\rm ff}\rangle}}$
$\newcommand{\tff}{\mbox{\langle\tau_{\rm ff}\rangle}}$
$\newcommand{\sigmol}{\mbox{\langle\Sigma_{\rm mol}\rangle}}$
$\newcommand{\tdep}{\mbox{\tau_{\rm dep}^{\rm mol}}}$
$\newcommand{\vect}[1]{\boldsymbol{#1}}$
$\newcommand\sol{\odot}$</div>



<div id="title">

# Reconciling extragalactic star formation efficiencies with theory: insights from PHANGS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.19832-b31b1b.svg)](https://arxiv.org/abs/2505.19832)<mark>Appeared on: 2025-05-27</mark> -  _Accepted for publication in A&A, 28 pages, 8 figures_

</div>
<div id="authors">

S. E. Meidt, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>A. Hughes</mark>

</div>
<div id="abstract">

**Abstract:** New extragalactic measurements of the cloud population-averaged star formation efficiency per free-fall time $\rm \epsilon_{\rm ff}$ from PHANGS show little sign of a theoretically predicted dependence on the gas virial level and weak variation with cloud-scale gas velocity dispersion.We explore ways to bring theory into consistency with the observations, in particular highlighting systematic variations in internal density structure that must happen together with an increase in virial parameter typical towards denser galaxy centers.To introduce these variations into conventional turbulence-regulated star formation models, we adopt three adjustments, together motivated by the expectation that the background host galaxy has an influence on the cloud-scale: we incorporate self-gravity and an internal density distribution that contains a broad power-law (PL) component and resembles the structure observed in local resolved clouds, we allow that the internal gas kinematics include motion in the background potential and let this regulate the onset of self-gravitation, and we assume that the distribution of gas densities is in a steady-state for only a fraction of a cloud free-fall time.  In practice, these changes significantly reduce the efficiencies predicted in multi-free-fall (MFF) scenarios compared to purely lognormal (LN) probability density functions (PDFs) and ties efficiency variations to variations in the slope of the PL $\alpha$ .We fit the model to PHANGS measurements of $\epsilon_{\rm ff}$ to identify the PL slopes that yield an optimal match.  These slopes vary systematically with galactic environment in the sense that gas that sits furthest from virial balance contains fractionally more gas at high density.  We relate this to the equilibrium response of gas in the presence of the galactic gravitational potential, which forces more gas to high density than characteristic of fully self-gravitating clouds.Viewing the efficiency variations as originating with time evolution in the power-law slope, our findings would alternatively imply coordination of the cloud evolutionary stage within environment.With this `galaxy regulation' behavior included, our preferred `self-gravitating' sgMFF models function similarly to the original, roughly `virialized cloud' single-free-fall models.  However, outside the environment of disks with their characteristic regulation, the flexible MFF models may be better suited.

</div>

<div id="div_fig1">

<img src="tmp_2505.19832/./outplot2asingle.png" alt="Fig7" width="100%"/>

**Figure 7. -** Diagnostics of the hybrid LN+PL density PDFs that match different MFF (and SFF) SF models to the $\rm \epsilon_{\rm ff}$ values measured in PHANGS, from left to right: the slope of the PL component of the PDF $\alpha$, the dense gas fraction $f_{\rm d}$ measured above a fixed density threshold (see text) and the self-gravitating fraction $f_{\rm sg}$ measured above the critical density $s_{\rm crit}$. The top row shows results for PDFs in the fiducial broad PL, shortened duration model given in Eq. (\ref{eq:finaleff}), adopting a MFF scenario. The second row shows results for PDFs in the broad PL, full duration model with $t_{\rm obs}=t_{\rm stop}$ in Eq. (\ref{eq:finaleff}) also in the MFF scenario.  The bottom two rows show MFF (third row) or SFF (fourth row) predictions adopting hybrid smooth-PDFs in the Burkhart formulation, with $s_{\rm t}=s_{\rm crit}$ advocated by \cite{burkmocz}.  All SF models shown here have assumed $\epsilon_{\rm core}=0.5$, $\phi_t=1$, $b=0.87$ and $c_s=0.3$ km s$^{-1}$ and the critical density given in Eq. (\ref{eq:scritKM05}).
As indicated by the color bar in the top right panel, points are color-coded by galactocentric radius (increasing outward, from red to purple). Symbol style denotes relative gas surface density; in all radial bins, regions sitting below the 30$^{th}$ percentile are marked with diamonds, regions between the 30$^{th}$ and 70$^{th}$ are marked with a circle, and regions above the 70$^{th}$ percentile are marked with a cross.  The gray points in the bottom row repeat the measurements from the top row.  The horizontal gray line (band) in the top left panel shows the mean (full spread) of the PL slopes $\alpha$ observed in local clouds \citep{Kainulainen14}.  The gray band in the middle panels illustrates the relation between dense gas fraction and gas surface density fitted by \cite{neumann23}. The width of the band represents the full spread in the fitted data (roughly 3 times the scatter about the fit line).
 (*fig:results1*)

</div>
<div id="div_fig2">

<img src="tmp_2505.19832/./PHANGSplot.jpeg" alt="Fig4" width="100%"/>

**Figure 4. -** The time-average $\rm \epsilon_{\rm ff}$ measured in 1.5-kpc wide hexagonal apertures sampling throughout 67 nearby galaxies targeted by PHANGS, as measured by \cite{leroy24}.  Measurements are plotted against the average cloud-scale velocity dispersion $\langle\sigma_{\rm mol}^{\rm cloud}\rangle$ in each aperture (\citealt{leroy24}; see \citealt{sun22,sun23} for details). Representative values for the $z\sim1$ clumps examined by \cite{dz23} are indicated by the thick gray point and bar, which depict the mean and rms of clump velocity dispersions, respectively, at the 11\% efficiency estimated for these clumps \citep{dz23}.
Colored lines illustrate basic predictions from turbulence-regulated SF models in single-free-fall (SFF) and multi-free-fall  (MFF) scenarios: \cite{KM05}(KM05, SFF; blue), \cite{PN11}(PN11, SFF; purple, scaled down by a factor of 10) and \cite{FK12}(FK12, MFF; red, scaled down by a factor of 100).  Each model is shown at three values of the virial parameter $\alpha_{\rm vir}=2.9$, $\alpha_{\rm vir}=5.3$ and $\alpha_{\rm vir}=9.7$, the 16$^{\rm th}$, 50$^{\rm th}$ and 84$^{\rm th}$ percentiles of the plotted regions, respectively.  The linewidth of the trends increases with increasing virial parameter.
All models adopt $\epsilon_{\rm core} =0.5$, $b=0.87$, $\phi_t=1.9$ and gas sound speed $c_s = 0.3   {\rm km   s^{-1}}$(see main text for definitions of these quantities) and, for comparison purposes, the same definition for the critical density, given by Eq. (\ref{eq:scritKM05}), estimated using the appropriate virial parameter for each line.
  (*fig:PHANGSsfe*)

</div>
<div id="div_fig3">

<img src="tmp_2505.19832/./outplot4.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
 Variation in the PL slope $\alpha$ calculated with the shortened duration model given in Eq. (\ref{eq:finaleff}) as a function of the cloud-scale virial parameter $\alpha_{\rm vir}$.  Symbol shapes and colors are as in Figure \ref{fig:results1}.
 The horizontal gray line (band) shows the mean (full spread) of the PL slopes $\alpha$ observed in local clouds \citep{Kainulainen14}. The reference line given by Eq. (\ref{eq:approximatealpha}) with $\mathcal{R}_\alpha=0.23\pm0.06$ is shown in black. Combining the systematic variation of dense gas fraction with virial state implied here together with variations in the critical density (depending on $\alpha_{\rm vir}$) produces star formation efficiencies that match the PHANGS observations in Fig. \ref{fig:PHANGSsfe}(see also Fig. \ref{fig:PHANGSsfeMODEL} and the discussion in $\S$\ref{sec:discussionMFFvKM05}).
  (*fig:alphaAlphavir*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.19832"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mb}[1]{{\color{red}{\bf} #1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\teff}{\textit{T}_{\text{eff}}}$
$\newcommand{\teffsun}{\textit{T}_{\text{eff},\bigodot}}$
$\newcommand{\logg}{\text{log}(\textit{g})}$
$\newcommand{\mh}{[\text{M}/\text{H}]}$
$\newcommand{\ofe}{[\text{O}/\text{Fe}]}$
$\newcommand{\feh}{[\text{Fe}/\text{H}]}$
$\newcommand{\mgfe}{[\text{Mg}/\text{Fe}]}$
$\newcommand{\sife}{[\text{Si}/\text{Fe}]}$
$\newcommand{\alpham}{[\alpha/\text{M}]}$
$\newcommand{\alphafe}{[\alpha/\text{Fe}]}$
$\newcommand{\nife}{[\text{Ni}/\text{Fe}]}$
$\newcommand{\cefe}{[\text{Ce}/\text{Fe}]}$
$\newcommand{\tife}{[\text{Ti}/\text{Fe}]}$
$\newcommand{\sfe}{[\text{S}/\text{Fe}]}$
$\newcommand{\crfe}{[\text{Cr}/\text{Fe}]}$
$\newcommand{\cafe}{[\text{Ca}/\text{Fe}]}$
$\newcommand{\nfe}{[\text{N}/\text{Fe}]}$
$\newcommand{\snr}{S/N}$
$\newcommand{\vrad}{V_{\text{rad}}}$
$\newcommand{\cms}{cm~s^{-2}}$
$\newcommand{\g}{G}$
$\newcommand{\bp}{\text{B}_\text{p}}$
$\newcommand{\rp}{\text{R}_\text{p}}$</div>



<div id="title">

# Impact of $\langle$3D$\rangle$ NLTE on GCE of oxygen\\with the RAdial Velocity Experiment

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.19875-b31b1b.svg)](https://arxiv.org/abs/2505.19875)<mark>Appeared on: 2025-05-27</mark> -  _11 pages, 12 figures. Accepted for publication in A&A_

</div>
<div id="authors">

<mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** The stellar chemical abundances, coupled with stellar kinematicsare a unique way to understand the chemo-dynamical processes that occurs tobuild the Milky Way and its local volume as we observe today. However, measuring stellar abundances is challenging as one needs to properlyaddress the effect of departure from the Local Thermodynamic Equilibrium (LTE), aswell as the commonly used 1-dimensional model atmosphere. In this work,we constrain the chemical evolution of [ O/Fe ] in FG stars of the RAdialVelocity Experiment (RAVE) with [ O/Fe ] abundances derived in non-LTE (NLTE)and with horizontally- temporally -averaged 3D ( $\langle$ 3D $\rangle$ ) model atmospheres. Using standard spectral fitting method, we determine for the first timeLTE and NLTE abundances of oxygen from the $\ion{O}{I}$ triplet at 8 446 Å inturn-off and dwarf stars thanks to intermediate-resolution RAVE spectra,assuming both 1D and ( $\langle$ 3D $\rangle$ ) model atmosphere. We find that NLTE effects plays a significant role when determining oxygeneven at a resolution of R= 7500. Typical NLTE-LTE corrections of the order of-0.12 dex are measured in dwarfs and turn-off stars using 1D MARCS models. In addition, $\langle$ 3D $\rangle$ modelling significantly impacts the oxygen abundance measurements.In contrast to applying $\langle$ 3D $\rangle$ NLTE abundance corrections or the classical1D LTE, the full $\langle$ 3D $\rangle$ NLTE spectral fitting yields improving the precisionof abundances by nearly 10 \% . We also showthat the decrease of [ O/Fe ] in the super-solar metallicity regime is rather characterisedby a flat trend when [ O/Fe ] is computed in $\langle$ 3D $\rangle$ NLTEfrom full spectral fitting. We attribute thisflattening at super-solar [ Fe/H ] to the interplay between locally born stars with negative [ O/Fe ] and stars migrated from the inner MW regions with super-solar [ O/Fe ] , supportingthe complex chemo-dynamical history of the Solar neighbourhood. Our results are key for understanding the effects of $\langle$ 3D $\rangle$ and NLTE whenmeasuring oxygen abundances in intermediate-resolution spectra. NLTE effect should be takeninto account when confronting Galactic chemical evolution models to observations. Thiswork is a test bed for the spectral analysis of the 4MOST low-resolutionspectra that will share similar properties as RAVE spectra in the redwavelength domain.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.19875"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

501  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
